### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [4]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.59069758 0.61548118 0.61714213 0.61501391 0.6095721  0.58830176
 0.61312382 0.61160496 0.61177751 0.61059543 0.58100469 0.59440182
 0.60920285 0.60206084 0.60475545 0.55540291 0.57867575 0.58594295
 0.59457236 0.58953558 0.6107016  0.62705003 0.62998005 0.63982436
 0.636084          nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [70 58], Predictions: [0 1], [65 63]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [70 58], Predictions: [0 1], [74 54]
Training RF ...
Model RF, Actual: [0 1], [70 58], Predictions: [0 1], [74 54]
Training XGB ...
Model XGB, Actual: [0 1], [70 58], Predictions: [0 1], [94 34]

None


[('mean_SCL', 166), ('bpm', 144), ('rmssd', 136), ('sdnn', 132), ('hf_rr', 102), ('lf_rr', 102), ('SCR_rate', 102)]


[('mean_SCL', 0.16751914319823222), ('lf_rr', 0.16050451109060385), ('hf_rr', 0.1400

One or more of the test scores are non-finite: [0.5854194  0.57836892 0.57883243 0.57971856 0.58310871 0.54929985
 0.58623502 0.58956966 0.58327869 0.57589046 0.56259968 0.55656191
 0.56415945 0.57264716 0.57022642 0.5585174  0.56237971 0.5732951
 0.56975547 0.58087391 0.55821191 0.55373215 0.55835796 0.56217072
 0.57065732        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 1, 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [70 59], Predictions: [0 1], [82 47]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [70 59], Predictions: [0 1], [99 30]
Training RF ...
Model RF, Actual: [0 1], [70 59], Predictions: [0 1], [85 44]
Training XGB ...
Model XGB, Actual: [0 1], [70 59], Predictions: [0 1], [103  26]

None


[('mean_SCL', 127), ('bpm', 97), ('hf_rr', 85), ('rmssd', 82), ('lf_rr', 59), ('sdnn', 55), ('SCR_rate', 51)]


[('rmssd', 0.16915588822796412), ('mean_SCL', 0.1516838843342905), ('lf_rr', 0.1484

...

Grid search for XGB ...
SVM: {'C': 1, 'gamma': 1, 'kernel': 'sigmoid'}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [34 66], Predictions: [0 1], [77 23]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 66], Predictions: [0 1], [60 40]
Training RF ...
Model RF, Actual: [0 1], [34 66], Predictions: [0 1], [58 42]
Training XGB ...
Model XGB, Actual: [0 1], [34 66], Predictions: [0 1], [68 32]

None


[('mean_SCL', 185), ('rmssd', 154), ('bpm', 143), ('sdnn', 108), ('hf_rr', 105), ('lf_rr', 99), ('SCR_rate', 84)]


[('rmssd', 0.1714462036586306), ('mean_SCL', 0.15293184559496986), ('lf_rr', 0.147

One or more of the test scores are non-finite: [0.53843159 0.55134223 0.56066412 0.56047662 0.55970174 0.513828
 0.54898794 0.55915679 0.56155087 0.56262633 0.51796882 0.54630773
 0.5485801  0.54381494 0.55048797 0.55066455 0.5630939  0.54991044
 0.5540302  0.56429361 0.56445482 0.57650355 0.56666136 0.57177078
 0.57801452        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 40, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [51 74], Predictions: [0 1], [86 39]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [51 74], Predictions: [0 1], [75 50]
Training RF ...
Model RF, Actual: [0 1], [51 74], Predictions: [0 1], [75 50]
Training XGB ...
Model XGB, Actual: [0 1], [51 74], Predictions: [0 1], [58 67]

None


[('mean_SCL', 336), ('bpm', 265), ('rmssd', 261), ('lf_rr', 244), ('hf_rr', 216), ('sdnn', 208), ('SCR_rate', 178)]


[('rmssd', 0.16400880019147945), ('mean_SCL', 0.1540235303484062), ('bpm', 0.15

One or more of the test scores are non-finite: [0.53592688 0.5574262  0.57440907 0.56624607 0.56593323 0.51819999
 0.53459976 0.55114369 0.56684273 0.5613812  0.58568828 0.57024501
 0.58026106 0.59104907 0.58307799 0.5208352  0.5397509  0.54527026
 0.5492454  0.55504853 0.5184183  0.54429373 0.56806934 0.58446715
 0.59721461        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid'}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [106  33], Predictions: [0 1], [131   8]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [106  33], Predictions: [0 1], [76 63]
Training RF ...
Model RF, Actual: [0 1], [106  33], Predictions: [0 1], [67 72]
Training XGB ...
Model XGB, Actual: [0 1], [106  33], Predictions: [0 1], [84 55]

None


[('mean_SCL', 236), ('bpm', 221), ('rmssd', 218), ('sdnn', 213), ('lf_rr', 210), ('hf_rr', 193), ('SCR_rate', 157)]


[('mean_SCL', 0.1674131040426149), ('hf_rr', 0.145828716637589